# Se inicia con la limpieza de los valores nan

In [53]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
import unidecode

In [54]:
# se trae un dataframe que contine la ubicacion de las viviendas
# !git clone git@github.com:juanguzman2/Precio-de-viviendas-en-colmbia.git

esta base de datos tambien fue obtenida pr los estudiantes

In [55]:
dfubi=pd.read_csv(r'C:\Users\Juan Guzman\pviviendas\Data\davipola.csv')
dfubi.head(1)

,Código Departamento,Código Municipio,Nombre Departamento,Nombre Municipio,Tipo: Municipio / Isla / Área no municipalizada
0,05,5001.0,ANTIOQUIA,MEDELLÍN,Municipio


In [56]:
# importacion dataset limpio de columnas
dfdata=pd.read_csv(r'C:\Users\Juan Guzman\pviviendas\Data\dfdatac')
dfdata.head(1)

,tipo,url,titulo,precio,area,habitaciones,banos,garajes,descripcion,Balcón,...,Baño Auxiliar,Zona de lavandería,Zonas Verdes,Supermercados / C.Comerciales,Instalación de gas,Parqueadero Visitantes,Colegios / Universidades,Parques cercanos,Zona Residencial,Closet
0,Usada,https://www.fincaraiz.com.co/casa-campestre-en...,Casa Campestre en Venta Rionegro Vereda Guayabito,$ 1.380.000.000,"320,00 m²",5,6,2,Propiedad moderna con muy buenos acabados. Her...,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [57]:
dfdata.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2715 entries, 0 to 2714
Data columns (total 24 columns):
 #   Column                         Non-Null Count  Dtype  
---  ------                         --------------  -----  
 0   tipo                           2715 non-null   object 
 1   url                            2715 non-null   object 
 2   titulo                         2715 non-null   object 
 3   precio                         2715 non-null   object 
 4   area                           2715 non-null   object 
 5   habitaciones                   2715 non-null   object 
 6   banos                          2715 non-null   object 
 7   garajes                        2715 non-null   object 
 8   descripcion                    2715 non-null   object 
 9   Balcón                         1401 non-null   float64
 10  Trans. Público cercano         1604 non-null   float64
 11  estrato                        2635 non-null   object 
 12  antiguedad                     2188 non-null   o

In [58]:
dfubi.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1126 entries, 0 to 1125
Data columns (total 5 columns):
 #   Column                                           Non-Null Count  Dtype  
---  ------                                           --------------  -----  
 0   Código Departamento                              1126 non-null   object 
 1   Código Municipio                                 1121 non-null   float64
 2   Nombre Departamento                              1121 non-null   object 
 3   Nombre Municipio                                 1121 non-null   object 
 4   Tipo: Municipio / Isla / Área no municipalizada  1121 non-null   object 
dtypes: float64(1), object(4)
memory usage: 44.1+ KB


## Cruce de la ifnormacion

In [51]:
# Aplicar limpieza a los nombres de las columnas para quitar caracteres extraños y evitar errores
lista_columnas=list(dfubi.columns)
col_limpia=[]

import unidecode

for columna in lista_columnas:
  col=columna.lower().replace('/','').replace('.','').strip().replace('  ',' ').replace('  ',' ').replace(' ', '_')
  col=unidecode.unidecode(col)
  col_limpia.append(col)

columnas_nuevas=dict(zip(lista_columnas,col_limpia))

In [61]:
dfubi=dfubi[dfubi["Nombre Departamento"]=='ANTIOQUIA']


In [62]:
#Aplicar transformación a los datos de municipios para usar el nombre como id de referencia en ambos datasets
dfubi["Nombre Municipio"]=dfubi["Nombre Municipio"].apply(lambda x: unidecode.unidecode(x.lower()))
 

In [63]:
list_muni=np.array(dfubi["Nombre Municipio"])
list_muni=np.append(list_muni, np.array(['ceja','viboral','llano','llanogrande','pereira','union','unión','santuario']))


In [64]:
#Funcion para extraer la ubicación de la propiedad a partir de los campos del dataset
def extract_ubicacion(lineas):
  texto=np.array(lineas)[0]
  palabras=texto.lower().replace(',','').replace(';','').split(' ')
  muni=''
  
  for palabra in palabras:

    if palabra in list_muni:
        muni = palabra
        break
    else:
        muni = ''

  if muni == '':
    texto=np.array(lineas)[1]
    palabras=texto.lower().replace(',','').replace(';','').split(' ')
  
    for palabra in palabras:

      if palabra in list_muni:
          muni = palabra
          break
      else:

          muni = ''


  return muni

In [65]:
#Funcion para extraer el tipo de propiedad a partir de los campos del dataset
tipo_propiedades=['casa','finca','apartamento','apartaestudio','lote', 'apartamentos']

def extract_tipo(lineas):

  texto=np.array(lineas)[0]

  palabras=texto.lower().split(' ')
  tipo_prop=''
  
  for palabra in palabras:

    if palabra in tipo_propiedades:
        tipo_prop = palabra
        break
    else:
        tipo_prop = ''

  if tipo_prop == '':

    texto=np.array(lineas)[1]

    palabras=texto.lower().split(' ')
    tipo_prop=''
    
    for palabra in palabras:

      if palabra in tipo_propiedades:
          tipo_prop = palabra
          break
      else:
          #tipo de propiedad por defecto será apartamento
          tipo_prop = 'apartamento'

  return tipo_prop

In [66]:
dfdata["municipio"]=dfdata[["titulo","descripcion"]].apply(extract_ubicacion,axis=1)

In [67]:
dfdata["tipo_propiedad"]=dfdata[["titulo","descripcion"]].apply(extract_tipo, axis=1)

In [68]:
dfdata.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2715 entries, 0 to 2714
Data columns (total 26 columns):
 #   Column                         Non-Null Count  Dtype  
---  ------                         --------------  -----  
 0   tipo                           2715 non-null   object 
 1   url                            2715 non-null   object 
 2   titulo                         2715 non-null   object 
 3   precio                         2715 non-null   object 
 4   area                           2715 non-null   object 
 5   habitaciones                   2715 non-null   object 
 6   banos                          2715 non-null   object 
 7   garajes                        2715 non-null   object 
 8   descripcion                    2715 non-null   object 
 9   Balcón                         1401 non-null   float64
 10  Trans. Público cercano         1604 non-null   float64
 11  estrato                        2635 non-null   object 
 12  antiguedad                     2188 non-null   o

## arreglo de algunas columnas

In [70]:
dfdata['municipio'].value_counts()


rionegro       1360
retiro          474
ceja            368
guarne          212
viboral          93
marinilla        90
llano            53
pereira          43
santuario        13
llanogrande       5
unión             4
Name: municipio, dtype: int64

In [72]:
#unificar apartamentos
dfdata['tipo_propiedad']=dfdata['tipo_propiedad'].apply(lambda x: x.replace("apartamentos", "apartamento"))


In [73]:
def unifica_muni(texto):
  muni=''
  
  if texto=='retiro':
    muni="el retiro"

  elif texto=='ceja':
    muni="la ceja"
 
  elif texto=='viboral':
    muni="el carmen de viboral"
  
  elif texto=='llano':
    muni="llanogrande"

  elif texto=='unión':
    muni="la unión"

  elif texto=='pereira':
    muni="san antonio de pereira"

  else:
    return texto

  return muni
     

In [76]:
dfdata['municipio']=dfdata['municipio'].apply(unifica_muni)
dfdata['municipio'].value_counts()


rionegro                  1360
el retiro                  474
la ceja                    368
guarne                     212
el carmen de viboral        93
marinilla                   90
llanogrande                 58
san antonio de pereira      43
santuario                   13
la unión                     4
Name: municipio, dtype: int64

In [77]:
dfdata.shape

(2715, 26)

In [78]:
dfdata.to_csv('dfmerged',index=False)